In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import warnings 
warnings.filterwarnings(action='ignore')
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re

In [ ]:
#파일 불러오기
train = pd.read_csv('/content/drive/My Drive/deepL/natural/train.csv', encoding = 'utf-8')
test = pd.read_csv('/content/drive/My Drive/deepL/natural/test_x.csv', encoding = 'utf-8')
sample_submission = pd.read_csv('/content/drive/My Drive/deepL/natural/sample_submission.csv', encoding ='utf-8')

In [ ]:
# train data 살펴보기
train

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3
...,...,...,...
54874,54874,"“Is that you, Mr. Smith?” odin whispered. “I h...",2
54875,54875,"I told my plan to the captain, and between us ...",4
54876,54876,"""Your sincere well-wisher, friend, and sister...",1
54877,54877,“Then you wanted me to lend you money?”,3


In [ ]:
#test 데이터 살펴보기
test

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...
...,...,...
19612,19612,"At the end of another day or two, odin growing..."
19613,19613,"All afternoon we sat together, mostly in silen..."
19614,19614,"odin, having carried his thanks to odin, proc..."
19615,19615,"Soon after this, upon odin's leaving the room,..."


In [ ]:
#sample_submission
sample_submission

,index,0,1,2,3,4
0,0,0,0,0,0,0
1,1,0,0,0,0,0
2,2,0,0,0,0,0
3,3,0,0,0,0,0
4,4,0,0,0,0,0
...,...,...,...,...,...,...
19612,19612,0,0,0,0,0
19613,19613,0,0,0,0,0
19614,19614,0,0,0,0,0
19615,19615,0,0,0,0,0


In [ ]:
#부호를 제거해주는 함수
def alpha_num(text):
    return re.sub(r'[^A-Za-z0-9 ]', '', text)
 
train['text']=train['text'].apply(alpha_num)

In [ ]:
#부호가 사라진 것을 확인할 수 있습니다.
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54874,54874,Is that you Mr Smith odin whispered I hardly d...,2
54875,54875,I told my plan to the captain and between us w...,4
54876,54876,Your sincere wellwisher friend and sister LUC...,1
54877,54877,Then you wanted me to lend you money,3


In [ ]:
# 이상치 제거
train = train[train.text != "     "]
train = train.reset_index(drop=True)
train

,index,text,author
0,0,He was almost choking There was so much so muc...,3
1,1,Your sister asked for it I suppose,2
2,2,She was engaged one day as she walked in peru...,1
3,3,The captain was in the porch keeping himself c...,4
4,4,Have mercy gentlemen odin flung up his hands D...,3
...,...,...,...
54831,54874,Is that you Mr Smith odin whispered I hardly d...,2
54832,54875,I told my plan to the captain and between us w...,4
54833,54876,Your sincere wellwisher friend and sister LUC...,1
54834,54877,Then you wanted me to lend you money,3


In [ ]:
# 불용어
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# 워드 클라우드를 통해 작가 별 공통적으로 다수 발견된 odin stopwords에 추가
stop_words=stopwords.words('english')
stop_words.append('odin')

In [ ]:
# 불용어 제거해주는 함수
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop_words:
            final_text.append(i.strip())
    return " ".join(final_text)
 
 
 
'''
# 불용어
stopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", 
             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", 
             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", 
             "have", "having", "he", "he'd", "he'll", "he's", "her", "here", "here's", "hers", "herself", "him", "himself", 
             "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in", "into", "is", "it", "it's", "its", "itself", 
             "let's", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", 
             "ourselves", "out", "over", "own", "same", "she", "she'd", "she'll", "she's", "should", "so", "some", "such", "than", "that", 
             "that's", "the", "their", "theirs", "them", "themselves", "then", "there", "there's", "these", "they", "they'd", "they'll", 
             "they're", "they've", "this", "those", "through", "to", "too", "under", "until", "up", "very", "was", "we", "we'd", "we'll", 
             "we're", "we've", "were", "what", "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", 
             "why", "why's", "with", "would", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves" ]
'''

'\n# 불용어\nstopwords = [ "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", \n             "at", "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "could", \n             "did", "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", \n             "have", "having", "he", "he\'d", "he\'ll", "he\'s", "her", "here", "here\'s", "hers", "herself", "him", "himself", \n             "his", "how", "how\'s", "i", "i\'d", "i\'ll", "i\'m", "i\'ve", "if", "in", "into", "is", "it", "it\'s", "its", "itself", \n             "let\'s", "me", "more", "most", "my", "myself", "nor", "of", "on", "once", "only", "or", "other", "ought", "our", "ours", \n             "ourselves", "out", "over", "own", "same", "she", "she\'d", "she\'ll", "she\'s", "should", "so", "some", "such", "than", "that", \n             "that\'s", "the", "their", "theirs", "them", "themselves", "then", "there", "there\

In [ ]:
# 어간추출
def stemming(text):
    final_text = []
    for i in text.split():
      final_text.append(stemmer.stem(i))
    return " ".join(final_text)

In [ ]:
#전처리 적용
train['text'] = train['text'].str.lower()
test['text'] = test['text'].str.lower()
train['text'] = train['text'].apply(alpha_num)#.apply(remove_stopwords)
test['text'] = test['text'].apply(alpha_num)#.apply(remove_stopwords)

In [ ]:
# train test 분리
X_train = np.array([x for x in train['text']])
X_test = np.array([x for x in test['text']])
y_train = np.array([x for x in train['author']])

In [ ]:
X_train

array(['he was almost choking there was so much so much he wanted to say but strange exclamations were all that came from his lips the pole gazed fixedly at him at the bundle of notes in his hand looked at odin and was in evident perplexity',
       'your sister asked for it i suppose',
       ' she was engaged one day as she walked in perusing janes last letter and dwelling on some passages which proved that jane had not written in spirits when instead of being again surprised by mr odin she saw on looking up that odin was meeting her putting away the letter immediately and forcing a smile she said',
       ..., ' your sincere wellwisher friend and sister lucy odin',
       'then you wanted me to lend you money',
       'it certainly had not occurred to me before but i said yes i should like that'],
      dtype='<U2441')

In [ ]:
#파라미터 설정
vocab_size = 20000
embedding_dim = 16
max_length = 500
padding_type='post'
#oov_tok = "<OOV>"

In [ ]:
#tokenizer에 fit
tokenizer = Tokenizer(num_words = vocab_size)#, oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [ ]:
word_index

In [ ]:
#데이터를 sequence로 변환해주고 padding 해줍니다.
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, padding=padding_type, maxlen=max_length)
 
test_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(test_sequences, padding=padding_type, maxlen=max_length)

In [ ]:
train_sequences

In [ ]:
train_padded

array([[    9,    11,   236, ...,     0,     0,     0],
       [   49,   322,   142, ...,     0,     0,     0],
       [   27,    11,   796, ...,     0,     0,     0],
       ...,
       [   49,  2525, 19858, ...,     0,     0,     0],
       [   66,    10,   422, ...,     0,     0,     0],
       [   12,   306,    16, ...,     0,     0,     0]], dtype=int32)

In [ ]:
# 다층 퍼셉트론 model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [ ]:
# compile model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
# model summary
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 16)           320000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 24)                408       
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 125       
Total params: 320,533
Trainable params: 320,533
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# fit model
num_epochs = 20
history = model.fit(train_padded, y_train, 
                    epochs=num_epochs, verbose=2, 
                    validation_split=0.2)

Epoch 1/20
1372/1372 - 6s - loss: 1.5505 - accuracy: 0.2799 - val_loss: 1.4940 - val_accuracy: 0.3667
Epoch 2/20
1372/1372 - 5s - loss: 1.3832 - accuracy: 0.4153 - val_loss: 1.2884 - val_accuracy: 0.4275
Epoch 3/20
1372/1372 - 5s - loss: 1.1691 - accuracy: 0.5282 - val_loss: 1.1100 - val_accuracy: 0.5520
Epoch 4/20
1372/1372 - 7s - loss: 1.0366 - accuracy: 0.5862 - val_loss: 1.0180 - val_accuracy: 0.5976
Epoch 5/20
Epoch 6/20
1372/1372 - 6s - loss: 0.8618 - accuracy: 0.6653 - val_loss: 0.8965 - val_accuracy: 0.6573
Epoch 7/20
1372/1372 - 6s - loss: 0.7950 - accuracy: 0.6962 - val_loss: 0.8689 - val_accuracy: 0.6694
Epoch 8/20
1372/1372 - 5s - loss: 0.7374 - accuracy: 0.7225 - val_loss: 0.8162 - val_accuracy: 0.6987
Epoch 9/20
1372/1372 - 6s - loss: 0.6875 - accuracy: 0.7446 - val_loss: 0.8033 - val_accuracy: 0.7033
Epoch 10/20
1372/1372 - 6s - loss: 0.6433 - accuracy: 0.7648 - val_loss: 0.7541 - val_accuracy: 0.7206
Epoch 11/20
1372/1372 - 6s - loss: 0.6060 - accuracy: 0.7806 - val_los

In [ ]:
# predict values
pred = model.predict_proba(test_padded)

Instructions for updating:
Please use `model.predict()` instead.


In [ ]:
pred

array([[3.2877051e-05, 9.5599562e-01, 3.8127601e-02, 5.8126464e-03,
        3.1364561e-05],
       [1.7886887e-01, 3.4912699e-01, 1.8379903e-01, 1.3160863e-02,
        2.7504420e-01],
       [9.9985516e-01, 7.1456692e-05, 5.8449263e-05, 4.7566672e-08,
        1.4936313e-05],
       ...,
       [1.0377860e-04, 9.9988210e-01, 8.4422345e-09, 1.4032576e-05,
        8.5060137e-09],
       [7.7965538e-05, 9.9991143e-01, 3.5215376e-06, 5.3770423e-06,
        1.6582266e-06],
       [9.9871945e-01, 4.6072760e-06, 6.7412818e-04, 1.2786379e-06,
        6.0056866e-04]], dtype=float32)

In [ ]:
# submission
sample_submission[['0','1','2','3','4']] = pred
sample_submission

,index,0,1,2,3,4
0,0,3.287705e-05,9.559956e-01,3.812760e-02,5.812646e-03,3.136456e-05
1,1,1.788689e-01,3.491270e-01,1.837990e-01,1.316086e-02,2.750442e-01
2,2,9.998552e-01,7.145669e-05,5.844926e-05,4.756667e-08,1.493631e-05
3,3,2.019325e-06,1.664023e-12,9.999971e-01,9.616202e-12,8.019470e-07
4,4,9.976664e-01,7.921504e-04,2.906237e-04,1.245494e-03,5.345343e-06
...,...,...,...,...,...,...
19612,19612,7.053748e-09,1.000000e+00,1.277726e-17,1.531390e-12,2.659210e-19
19613,19613,9.616914e-04,1.927661e-06,1.410670e-03,1.559642e-10,9.976258e-01
19614,19614,1.037786e-04,9.998821e-01,8.442234e-09,1.403258e-05,8.506014e-09
19615,19615,7.796554e-05,9.999114e-01,3.521538e-06,5.377042e-06,1.658227e-06


In [ ]:
sample_submission.to_csv('submission.csv', index = False, encoding = 'utf-8')